In [ ]:
"""
================================================================================
Phase 02B: Per-Ticker Merging of EOD Historical Options Chain Data
================================================================================

Script Name: Phase_02B_merge_eod_chain.py
Pipeline Position: Phase 02B sits between Phase 02A (underlying OHLCV merge) and 
                   feature engineering (Phase 03+), completing the data enrichment 
                   pipeline by adding options-specific market data.

--------------------------------------------------------------------------------
PURPOSE
--------------------------------------------------------------------------------
This module enriches options trade data (already merged with underlying OHLCV 
from Phase 02A) with EOD historical options chain data from Intrinio. It joins 
intraday options trades with per-contract EOD data including open interest, 
implied volatility, and Greeks (delta, gamma, theta, vega) for downstream 
anomaly detection analysis.

Key Operations:
  1. Per-Ticker Processing — Processes trades grouped by underlying ticker for 
     efficient memory usage and targeted output files
  2. EOD Chain Data Enrichment — Maps current-day open interest, implied 
     volatility, and Greeks to each trade
  3. Previous Day Open Interest — Retrieves open interest from the previous 
     trading day (accounting for weekends and holidays via NYSE calendar)
  4. Ticker Format Conversion — Converts between EOD chain format 
     (e.g., AAPL240315C00075000) and OCC format (e.g., O:AAPL240315C00075000)
  5. Data Export — Saves enriched trade data as per-ticker, date-partitioned 
     parquet files

--------------------------------------------------------------------------------
DIRECTORY STRUCTURE
--------------------------------------------------------------------------------
project_root/
│
├── Phase_02B_merge_eod_chain.py        # This script
│
├── input/
│   ├── TRADES_WITH_UNDERLYING/         # Output from Phase 02A (input)
│   │   ├── 2024-02-01.parquet
│   │   ├── 2024-02-02.parquet
│   │   └── ...
│   │
│   └── EOD_OPTIONS_CHAIN/              # EOD chain data from Intrinio (FLAT)
│       ├── AAPL_2024-02-01.csv         # NOT nested in subfolders!
│       ├── TSLA_2024-02-01.csv
│       └── ...
│
└── output/                             # Enriched trades data (output)
    ├── AAPL_comptrades_2024-02-01.parquet
    ├── TSLA_comptrades_2024-02-01.parquet
    └── ...

--------------------------------------------------------------------------------
INPUT SPECIFICATIONS
--------------------------------------------------------------------------------
1. Options Trades Data (trades_input_path/)
   - Filename Format: YYYY-MM-DD.parquet (output from Phase 02A)
   - Required Columns:
     * ticker      — Option contract ticker in OCC format (e.g., O:AAPL240315C00075000)
     * underlying  — Underlying stock ticker symbol (e.g., AAPL, TSLA)

2. EOD Options Chain Data (eod_chain_input_path/)
   - Filename Format: {TICKER}_{DATE}.csv (FLAT structure, NOT nested)
   - Example: AAPL_2024-02-01.csv
   - Required Columns:
     * code               — Option contract code (e.g., AAPL240315C00075000)
                            NOTE: Use 'code' column, NOT 'ticker' column!
     * open_interest      — Total open contracts
     * implied_volatility — IV calculated using Black-Scholes
     * delta              — Rate of change vs $1 underlying move
     * gamma              — Rate of change of delta
     * theta              — Time decay
     * vega               — Sensitivity to IV changes

--------------------------------------------------------------------------------
OUTPUT SPECIFICATIONS
--------------------------------------------------------------------------------
- Filename Format: {TICKER}_comptrades_YYYY-MM-DD.parquet
- Example: AAPL_comptrades_2024-02-01.parquet

New columns added to output:
  * open_interest_now       — EOD open interest for the trade date
  * open_interest_yesterday — EOD open interest for previous trading day
  * implied_volatility      — EOD implied volatility
  * grk_delta               — EOD delta
  * grk_gamma               — EOD gamma
  * grk_theta               — EOD theta
  * grk_vega                — EOD vega
  * trade_date              — Trade date in YYYY-MM-DD format

Note: If an option contract is not found in EOD chain data, enrichment 
columns will be NaN.

--------------------------------------------------------------------------------
CONFIGURATION
--------------------------------------------------------------------------------
Modify the CONFIG dictionary:

CONFIG = {
    "start_date": "2024-02-01",           # Start date (YYYY-MM-DD)
    "end_date": "2024-12-31",             # End date (YYYY-MM-DD)
    "tickers": ["AAPL", "TSLA"],          # List of tickers, or None for all
    "trades_input_path": "./input/TRADES_WITH_UNDERLYING",
    "eod_chain_input_path": "./input/EOD_OPTIONS_CHAIN",
    "output_path": "./output",
}

Parameters:
  * start_date          — First date to process (inclusive)
  * end_date            — Last date to process (inclusive)
  * tickers             — List of underlying tickers, or None to process all
  * trades_input_path   — Path to Phase 02A output (parquet files)
  * eod_chain_input_path— Path to EOD options chain CSV files (FLAT structure)
  * output_path         — Path for enriched output files

--------------------------------------------------------------------------------
USAGE
--------------------------------------------------------------------------------
Basic execution:
    python Phase_02B_merge_eod_chain.py

Process specific tickers for one day:
    CONFIG["tickers"] = ["AAPL", "TSLA", "NVDA"]
    CONFIG["start_date"] = "2024-02-01"
    CONFIG["end_date"] = "2024-02-01"

Process ALL tickers for a date range:
    CONFIG["tickers"] = None
    CONFIG["start_date"] = "2024-02-01"
    CONFIG["end_date"] = "2024-12-31"

Expected console output:
    Processing 252 trading days from 2024-02-01 to 2024-12-31
    ############################################################
    # DATE: 2024-02-01
    ############################################################
    Loaded 5,793,340 trades from .../2024-02-01.parquet
    ============================================================
    Processing AAPL for 2024-02-01
    ============================================================
    Found 12,487 trades for AAPL
    Loaded 2,358 chain records from .../AAPL_2024-02-01.csv
    Previous trading day: 2024-01-31
    EOD match rate: 11,892/12,487 trades (95.2%)
    Written 12,487 rows to .../AAPL_comptrades_2024-02-01.parquet

--------------------------------------------------------------------------------
DEPENDENCIES
--------------------------------------------------------------------------------
Required:
  * pandas                  — Data manipulation
  * numpy                   — Numerical operations
  * pyarrow                 — Parquet file I/O

Optional:
  * pandas_market_calendars — NYSE trading calendar for previous trading day
                              calculation (falls back to weekday logic if unavailable)

Install:
    pip install pandas numpy pyarrow pandas_market_calendars

--------------------------------------------------------------------------------
GITHUB LOCATION
--------------------------------------------------------------------------------
  * Owner: toolsandsoftware@cyclelabs.net
  * Repo:  20260201_UOAResearchPipeline
  * Path:  notebooks/002_DATAMERGING/02v1_merge_eod_options_chain.py
  * Link:  https://github.com/toolsandsoftware-cyclelabs/20260201_UOAResearchPipeline

================================================================================
"""

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import re
from pathlib import Path

# Try to import trading calendar - fallback to simple logic if unavailable
try:
    import pandas_market_calendars as mcal
    NYSE_CALENDAR = mcal.get_calendar('NYSE')
    HAS_MARKET_CALENDAR = True
except ImportError:
    HAS_MARKET_CALENDAR = False
    print("Warning: pandas_market_calendars not installed. Using fallback trading day logic.")
    print("Install with: pip install pandas_market_calendars")


# =============================================================================
# CONFIGURATION - MODIFY THESE AS NEEDED
# =============================================================================

CONFIG = {
    # -------------------------------------------------------------------------
    # DATE RANGE
    # -------------------------------------------------------------------------
    "start_date": "2023-12-04",      # Start date (YYYY-MM-DD)
    "end_date": "2023-12-04",        # End date (YYYY-MM-DD)
    
    # -------------------------------------------------------------------------
    # TICKERS TO PROCESS
    # -------------------------------------------------------------------------
    # Set to a list of underlying tickers to process specific ones:
    #   ["AAPL", "TSLA", "NVDA", "SPY"]
    # Set to None to process ALL underlyings found in the trades file:
    #   None
    "tickers": None,
    
    # -------------------------------------------------------------------------
    # INPUT PATHS (Local filesystem or mounted GCS paths)
    # -------------------------------------------------------------------------
    # Trades data: expects files named YYYY-MM-DD.parquet containing all trades
    "trades_input_path": "/path/to/trades",
    
    # EOD chain data: expects files named {TICKER}_{DATE}.csv in FLAT structure
    # Example: /path/to/eod/AAPL_2023-12-04.csv (NOT /path/to/eod/AAPL/AAPL_2023-12-04.csv)
    "eod_chain_input_path": "/path/to/eod/output",
    
    # -------------------------------------------------------------------------
    # OUTPUT PATH
    # -------------------------------------------------------------------------
    # Output files will be named: {ticker}_comptrades_YYYY-MM-DD.parquet
    # Example: AAPL_comptrades_2023-12-04.parquet
    "output_path": "/path/to/output",
}


# =============================================================================
# TRADING CALENDAR FUNCTIONS
# =============================================================================

def get_trading_days_in_range(start_date: str, end_date: str) -> list:
    """Get all trading days in a date range"""
    if HAS_MARKET_CALENDAR:
        schedule = NYSE_CALENDAR.schedule(start_date=start_date, end_date=end_date)
        return [d.strftime("%Y-%m-%d") for d in schedule.index]
    else:
        # Fallback: return all weekdays (won't account for holidays)
        dates = []
        current = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        while current <= end:
            if current.weekday() < 5:  # Monday = 0, Friday = 4
                dates.append(current.strftime("%Y-%m-%d"))
            current += timedelta(days=1)
        return dates


def get_previous_trading_day(date_str: str) -> str:
    """
    Get the previous trading day accounting for weekends and holidays.
    
    Args:
        date_str: Date in YYYY-MM-DD format
        
    Returns:
        Previous trading day in YYYY-MM-DD format
    """
    if HAS_MARKET_CALENDAR:
        # Look back up to 10 days to find previous trading day
        current_date = datetime.strptime(date_str, "%Y-%m-%d")
        lookback_start = (current_date - timedelta(days=10)).strftime("%Y-%m-%d")
        lookback_end = (current_date - timedelta(days=1)).strftime("%Y-%m-%d")
        
        schedule = NYSE_CALENDAR.schedule(start_date=lookback_start, end_date=lookback_end)
        if len(schedule) > 0:
            return schedule.index[-1].strftime("%Y-%m-%d")
        else:
            # Fallback if no trading days found
            return (current_date - timedelta(days=1)).strftime("%Y-%m-%d")
    else:
        # Fallback: skip weekends only
        current_date = datetime.strptime(date_str, "%Y-%m-%d")
        previous_date = current_date - timedelta(days=1)
        
        # Skip weekends
        while previous_date.weekday() >= 5:  # Saturday = 5, Sunday = 6
            previous_date -= timedelta(days=1)
        
        return previous_date.strftime("%Y-%m-%d")


# =============================================================================
# TICKER FORMAT CONVERSION
# =============================================================================

def convert_to_occ_format(ticker_str: str) -> str:
    """
    Convert EOD format to OCC format
    Example: A240315C00075000 -> O:A240315C00075000
    """
    if pd.isna(ticker_str):
        return ticker_str
    if not str(ticker_str).startswith("O:"):
        return f"O:{ticker_str}"
    return ticker_str


def extract_underlying_from_option_ticker(option_ticker: str) -> str:
    """
    Extract underlying ticker from OCC format option ticker
    Example: O:AAPL240315C00075000 -> AAPL
             O:A240315C00075000 -> A
    """
    if pd.isna(option_ticker):
        return None
    
    # Remove O: prefix if present
    ticker = str(option_ticker).replace("O:", "")
    
    # Find where the date starts (6 digits YYMMDD)
    match = re.match(r'^([A-Z]+)', ticker)
    if match:
        return match.group(1)
    return None


# =============================================================================
# FILE I/O FUNCTIONS
# =============================================================================

def read_trades_file(trades_path: str, date_str: str) -> pd.DataFrame:
    """
    Read trades parquet file for a specific date.
    Expected filename: YYYY-MM-DD.parquet (contains all tickers for that day)
    """
    filepath = os.path.join(trades_path, f"{date_str}.parquet")
    
    if not os.path.exists(filepath):
        print(f"Warning: Trades file not found - {filepath}")
        return None
    
    # Use pyarrow directly to avoid pandas extension type conflicts
    try:
        import pyarrow.parquet as pq
        table = pq.read_table(filepath)
        df = table.to_pandas()
    except Exception as e:
        print(f"PyArrow read failed ({e}), trying pandas directly...")
        df = pd.read_parquet(filepath, engine='pyarrow')
    
    print(f"Loaded {len(df):,} trades from {filepath}")
    return df


def read_eod_chain_file(eod_path: str, ticker: str, date_str: str) -> pd.DataFrame:
    """
    Read EOD chain CSV file for a specific ticker and date.
    Expected filename: {TICKER}_{DATE}.csv (flat structure)
    """
    filepath = os.path.join(eod_path, f"{ticker}_{date_str}.csv")
    
    if not os.path.exists(filepath):
        print(f"Warning: EOD chain file not found - {filepath}")
        return None
    
    df = pd.read_csv(filepath)
    print(f"Loaded {len(df):,} chain records from {filepath}")
    return df


def write_output_parquet(df: pd.DataFrame, output_path: str, ticker: str, date_str: str):
    """
    Write merged data to parquet file.
    Output filename: {ticker}_comptrades_YYYY-MM-DD.parquet
    """
    os.makedirs(output_path, exist_ok=True)
    filepath = os.path.join(output_path, f"{ticker}_comptrades_{date_str}.parquet")
    
    # Use pyarrow directly for more robust writing
    try:
        import pyarrow as pa
        import pyarrow.parquet as pq
        table = pa.Table.from_pandas(df, preserve_index=False)
        pq.write_table(table, filepath)
    except Exception as e:
        print(f"PyArrow write failed ({e}), trying pandas directly...")
        df.to_parquet(filepath, index=False, engine='pyarrow')
    
    print(f"Written {len(df):,} rows to {filepath}")
    return filepath


# =============================================================================
# MAIN MERGE FUNCTION
# =============================================================================

def merge_trades_with_eod(
    trades_df: pd.DataFrame,
    ticker: str,
    date_str: str,
    eod_path: str
) -> pd.DataFrame:
    """
    Merge trades for a specific underlying with EOD chain data.
    
    Only merges with actual trades - no phantom rows for untraded options.
    
    Args:
        trades_df: DataFrame containing trades for all underlyings
        ticker: Underlying ticker to process (e.g., 'AAPL')
        date_str: Trade date in YYYY-MM-DD format
        eod_path: Path to EOD chain files
        
    Returns:
        Merged DataFrame with trades enriched with EOD data
    """
    print(f"\n{'='*60}")
    print(f"Processing {ticker} for {date_str}")
    print(f"{'='*60}")
    
    # 1. Filter trades for this underlying
    if 'underlying' in trades_df.columns:
        ticker_trades = trades_df[trades_df['underlying'] == ticker].copy()
    else:
        # Extract underlying from option ticker if not present
        trades_df['_underlying'] = trades_df['ticker'].apply(extract_underlying_from_option_ticker)
        ticker_trades = trades_df[trades_df['_underlying'] == ticker].copy()
        ticker_trades = ticker_trades.drop(columns=['_underlying'])
    
    if len(ticker_trades) == 0:
        print(f"No trades found for {ticker} on {date_str}")
        return None
    
    print(f"Found {len(ticker_trades):,} trades for {ticker}")
    
    # 2. Load EOD chain data for current day
    eod_df = read_eod_chain_file(eod_path, ticker, date_str)
    
    if eod_df is None:
        print(f"No EOD chain data found for {ticker} on {date_str}")
        # Return trades without EOD enrichment
        ticker_trades['open_interest_now'] = np.nan
        ticker_trades['open_interest_yesterday'] = np.nan
        ticker_trades['implied_volatility'] = np.nan
        ticker_trades['grk_delta'] = np.nan
        ticker_trades['grk_gamma'] = np.nan
        ticker_trades['grk_theta'] = np.nan
        ticker_trades['grk_vega'] = np.nan
        ticker_trades['trade_date'] = date_str
        return ticker_trades
    
    # 3. Load EOD chain data for previous trading day (for open_interest_yesterday)
    prev_date = get_previous_trading_day(date_str)
    print(f"Previous trading day: {prev_date}")
    
    eod_prev_df = read_eod_chain_file(eod_path, ticker, prev_date)
    
    # 4. Convert ticker formats in EOD data to match trades (OCC format)
    # Note: EOD files use 'code' column for option contract symbols, not 'ticker'
    eod_df['ticker_occ'] = eod_df['code'].apply(convert_to_occ_format)

    if eod_prev_df is not None:
        eod_prev_df['ticker_occ'] = eod_prev_df['code'].apply(convert_to_occ_format)
        prev_oi_map = dict(zip(eod_prev_df['ticker_occ'], eod_prev_df['open_interest']))
    else:
        prev_oi_map = {}
        print(f"Warning: No previous day EOD data found for {ticker}")
    
    # 5. Create mapping dictionaries from EOD data
    eod_columns = ['open_interest', 'implied_volatility', 'delta', 'gamma', 'theta', 'vega']
    available_columns = [c for c in eod_columns if c in eod_df.columns]
    
    eod_map = eod_df.set_index('ticker_occ')[available_columns].to_dict('index')
    
    # 6. Map EOD data to trades
    ticker_trades['open_interest_now'] = ticker_trades['ticker'].map(
        lambda x: eod_map.get(x, {}).get('open_interest', np.nan)
    )
    
    ticker_trades['open_interest_yesterday'] = ticker_trades['ticker'].map(
        lambda x: prev_oi_map.get(x, np.nan)
    )
    
    ticker_trades['implied_volatility'] = ticker_trades['ticker'].map(
        lambda x: eod_map.get(x, {}).get('implied_volatility', np.nan)
    )
    
    ticker_trades['grk_delta'] = ticker_trades['ticker'].map(
        lambda x: eod_map.get(x, {}).get('delta', np.nan)
    )
    
    ticker_trades['grk_gamma'] = ticker_trades['ticker'].map(
        lambda x: eod_map.get(x, {}).get('gamma', np.nan)
    )
    
    ticker_trades['grk_theta'] = ticker_trades['ticker'].map(
        lambda x: eod_map.get(x, {}).get('theta', np.nan)
    )
    
    ticker_trades['grk_vega'] = ticker_trades['ticker'].map(
        lambda x: eod_map.get(x, {}).get('vega', np.nan)
    )
    
    # 7. Add trade date column
    ticker_trades['trade_date'] = date_str
    
    # 8. Report merge statistics
    matched = ticker_trades['open_interest_now'].notna().sum()
    total = len(ticker_trades)
    print(f"EOD match rate: {matched:,}/{total:,} trades ({100*matched/total:.1f}%)")
    
    return ticker_trades


# =============================================================================
# MAIN PROCESSING FUNCTION
# =============================================================================

def process_date_range(config: dict):
    """
    Process all tickers for a date range.
    
    Args:
        config: Configuration dictionary
    """
    start_date = config["start_date"]
    end_date = config["end_date"]
    tickers_to_process = config["tickers"]
    trades_path = config["trades_input_path"]
    eod_path = config["eod_chain_input_path"]
    output_path = config["output_path"]
    
    # Get trading days in range
    trading_days = get_trading_days_in_range(start_date, end_date)
    print(f"\nProcessing {len(trading_days)} trading days from {start_date} to {end_date}")
    
    output_files = []
    
    for date_str in trading_days:
        print(f"\n{'#'*60}")
        print(f"# DATE: {date_str}")
        print(f"{'#'*60}")
        
        # Load all trades for this date
        trades_df = read_trades_file(trades_path, date_str)
        
        if trades_df is None:
            print(f"Skipping {date_str} - no trades file found")
            continue
        
        # Determine which tickers to process
        if tickers_to_process is None:
            # Process all unique underlyings in the trades file
            if 'underlying' in trades_df.columns:
                available_tickers = trades_df['underlying'].dropna().unique().tolist()
            else:
                trades_df['_underlying'] = trades_df['ticker'].apply(extract_underlying_from_option_ticker)
                available_tickers = trades_df['_underlying'].dropna().unique().tolist()
            print(f"Found {len(available_tickers)} unique underlyings in trades data")
        else:
            available_tickers = tickers_to_process
        
        # Process each ticker
        for ticker in available_tickers:
            try:
                merged_df = merge_trades_with_eod(
                    trades_df=trades_df,
                    ticker=ticker,
                    date_str=date_str,
                    eod_path=eod_path
                )
                
                if merged_df is not None and len(merged_df) > 0:
                    output_file = write_output_parquet(merged_df, output_path, ticker, date_str)
                    output_files.append(output_file)
                    
            except Exception as e:
                print(f"Error processing {ticker} on {date_str}: {str(e)}")
                import traceback
                traceback.print_exc()
    
    print(f"\n{'='*60}")
    print(f"PROCESSING COMPLETE")
    print(f"{'='*60}")
    print(f"Total output files created: {len(output_files)}")
    
    return output_files


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    """
    USAGE EXAMPLES:
    
    1. Process specific tickers for one day:
       CONFIG["tickers"] = ["AAPL", "TSLA", "NVDA"]
       CONFIG["start_date"] = "2023-12-04"
       CONFIG["end_date"] = "2023-12-04"
    
    2. Process ALL tickers for a date range:
       CONFIG["tickers"] = None
       CONFIG["start_date"] = "2023-12-01"
       CONFIG["end_date"] = "2023-12-31"
    
    3. Process single ticker for a month:
       CONFIG["tickers"] = ["SPY"]
       CONFIG["start_date"] = "2023-12-01"
       CONFIG["end_date"] = "2023-12-29"
    
    INPUT FILE EXPECTATIONS:
    - Trades: {trades_input_path}/YYYY-MM-DD.parquet
    - EOD:    {eod_chain_input_path}/{TICKER}_{DATE}.csv  (FLAT, not nested!)
    
    OUTPUT:
    - {output_path}/{ticker}_comptrades_YYYY-MM-DD.parquet
    """
    
    # =========================================================================
    # CONFIGURE YOUR RUN HERE
    # =========================================================================
    CONFIG.update({
        # Date range
        "start_date": "2024-02-01",
        "end_date": "2025-12-31",
        
        # Tickers: list for specific tickers, None for all
        "tickers": ["CIFR"],
        
        # Input paths - UPDATE THESE FOR YOUR ENVIRONMENT
        "trades_input_path": r"D:\cyclelabs_codes\CL_20251120_siphontrades\01_FIXINGRAWDATA\output",
        "eod_chain_input_path": r"D:\cyclelabs_codes\CL_20260116_anomalyfixed\000_DATA01_EODHISTORICALOPTIONS",
        
        # Output path - UPDATE THIS
        "output_path": r"D:\cyclelabs_codes\CL_20251120_siphontrades\01_FIXINGRAWDATA\output_mergedall",
    })
    
    # Run the merge
    output_files = process_date_range(CONFIG)
    
    print(f"\nGenerated {len(output_files)} output files:")
    for f in output_files[:10]:
        print(f"  - {f}")
    if len(output_files) > 10:
        print(f"  ... and {len(output_files) - 10} more")